In [1]:
import pandas as pd

In [2]:
# 코스피 수익성, 성장성, 안정성, 활동성, 현금흐름, 베네시, 벤포드, 분식여부 csv 가져오기
kospi_profitability = pd.read_csv('./값채우기/코스피_수익성.csv', thousands=',')
kospi_growth = pd.read_csv('./값채우기/코스피_성장성.csv', thousands=',')
kospi_stability = pd.read_csv('./값채우기/코스피_안정성.csv', thousands=',')
kospi_activity = pd.read_csv('./값채우기/코스피_활동성.csv', thousands=',')
kospi_cf = pd.read_csv('./값채우기/코스피_현금흐름.csv', thousands=',')
kospi_beneish = pd.read_csv('./값채우기/코스피_베니시.csv', thousands=',')
kospi_benford = pd.read_csv('./값채우기/kospi_benford.csv')
kospi_company = pd.read_csv('./감리지적기업연도/코스피_분식기업.csv')

In [3]:
kospi_benford.drop(['결산월', '주기'], axis=1, inplace=True)

In [4]:
# 코스닥 수익성, 성장성, 안정성, 활동성, 현금흐름, 베네시, 벤포드, 분식여부 csv 가져오기
kosdaq_profitability = pd.read_csv('./값채우기/코스닥_수익성.csv', thousands=',')
kosdaq_growth = pd.read_csv('./값채우기/코스닥_성장성.csv', thousands=',')
kosdaq_stability = pd.read_csv('./값채우기/코스닥_안정성.csv', thousands=',')
kosdaq_activity = pd.read_csv('./값채우기/코스닥_활동성.csv', thousands=',')
kosdaq_cf = pd.read_csv('./값채우기/코스닥_현금흐름.csv', thousands=',')
kosdaq_beneish = pd.read_csv('./값채우기/코스닥_베니시.csv', thousands=',')
kosdaq_benford = pd.read_csv('./값채우기/kosdaq_benford.csv')
kosdaq_company = pd.read_csv('./감리지적기업연도/코스닥_분식기업.csv')

In [5]:
kosdaq_benford.drop(['결산월', '주기'], axis=1, inplace=True)

In [6]:
print(kospi_profitability.shape, 
      kospi_profitability.drop_duplicates('Name').shape, 
      kosdaq_profitability.shape, 
      kosdaq_profitability.drop_duplicates('Name').shape)

# 원데이터
# 코스피: 13356개 / 1100개 기업
# 코스닥: 27840개 / 1696개 기업

(13356, 11) (1100, 11) (27840, 11) (2317, 11)


---
### 이너조인, 데이터 병합시키기

In [7]:
kospi_all = pd.merge(kospi_profitability, kospi_growth, on=['Symbol', 'Name', '회계년'])
kospi_all = pd.merge(kospi_all, kospi_stability, on=['Symbol', 'Name', '회계년'])
kospi_all = pd.merge(kospi_all, kospi_activity, on=['Symbol', 'Name', '회계년'])
kospi_all = pd.merge(kospi_all, kospi_cf, on=['Symbol', 'Name', '회계년'])
kospi_all = pd.merge(kospi_all, kospi_beneish, on=['Symbol', 'Name', '회계년'])
kospi_all = pd.merge(kospi_all, kospi_benford, on=['Symbol', 'Name', '회계년'])
kospi_all = pd.merge(kospi_all, kospi_company, on=['Name', '회계년'], how='left')

In [8]:
kosdaq_all = pd.merge(kosdaq_profitability, kosdaq_growth, on=['Symbol', 'Name', '회계년'])
kosdaq_all = pd.merge(kosdaq_all, kosdaq_stability, on=['Symbol', 'Name', '회계년'])
kosdaq_all = pd.merge(kosdaq_all, kosdaq_activity, on=['Symbol', 'Name', '회계년'])
kosdaq_all = pd.merge(kosdaq_all, kosdaq_cf, on=['Symbol', 'Name', '회계년'])
kosdaq_all = pd.merge(kosdaq_all, kosdaq_beneish, on=['Symbol', 'Name', '회계년'])
kosdaq_all = pd.merge(kosdaq_all, kosdaq_benford, on=['Symbol', 'Name', '회계년'])
kosdaq_all = pd.merge(kosdaq_all, kosdaq_company, on=['Name', '회계년'], how='left')

In [9]:
import numpy as np
kospi_all['분식기업'].replace(np.nan, 0, inplace=True)
kosdaq_all['분식기업'].replace(np.nan, 0, inplace=True)

---
### 11년 데이터 제거

In [10]:
kospi_all = kospi_all[kospi_all['회계년'] != 2011]
kosdaq_all = kosdaq_all[kosdaq_all['회계년'] != 2011]

In [11]:
print(kospi_all.shape, 
      kospi_all.drop_duplicates('Name').shape, 
      kosdaq_all.shape, 
      kosdaq_all.drop_duplicates('Name').shape)

# 코스피: 7356개 / 733개 기업
# 코스닥: 12214개 / 1472개 기업

(12250, 46) (1100, 46) (25479, 46) (2313, 46)


---
### 상장 이전 회계연도 데이터 제거하기 (불필요한 데이터 제거)

In [12]:
# KRX에서 상장일 데이터 가져왔음! 불러오기!! (코스피)

kospi_info = pd.read_csv('./코스피_기본정보.csv', encoding='CP949')
kospi_info = kospi_info[['단축코드', '한글 종목약명', '상장일']]

# '상장일' 칼럼을 날짜 형식으로 변환
kospi_info['상장일'] = pd.to_datetime(kospi_info['상장일'], format='%Y/%m/%d')

# 연도만 남기기
kospi_info['상장일'] = kospi_info['상장일'].dt.year

# '상장일' 칼럼을 문자열로 변환
kospi_info['상장일'] = kospi_info['상장일'].astype(str)

# '단축코드' 칼럼의 모든 데이터에 'A'를 추가하기
kospi_info['단축코드'] = 'A' + kospi_info['단축코드'].astype(str)

# '단축코드' 칼럼명을 'Symbol'로 변경하기
kospi_info.rename(columns={'단축코드':'Symbol', 
                           '한글 종목약명':'Name', 
                           '상장일':'상장연도'}, inplace=True)

In [13]:
# KRX에서 상장일 데이터 가져왔음! 불러오기!! (코스닥)

kosdaq_info = pd.read_csv('./코스닥_기본정보.csv', encoding='CP949')
kosdaq_info = kosdaq_info[['단축코드', '한글 종목약명', '상장일']]

# '상장일' 칼럼을 날짜 형식으로 변환
kosdaq_info['상장일'] = pd.to_datetime(kosdaq_info['상장일'], format='%Y/%m/%d')

# 연도만 남기기
kosdaq_info['상장일'] = kosdaq_info['상장일'].dt.year

# '상장일' 칼럼을 문자열로 변환
kosdaq_info['상장일'] = kosdaq_info['상장일'].astype(str)

# '단축코드' 칼럼의 모든 데이터에 'A'를 추가하기
kosdaq_info['단축코드'] = 'A' + kosdaq_info['단축코드'].astype(str)

# '단축코드' 칼럼명을 'Symbol'로 변경하기
kosdaq_info.rename(columns={'단축코드':'Symbol', 
                            '한글 종목약명':'Name', 
                            '상장일':'상장연도'}, inplace=True)

In [14]:
kospi_all = pd.merge(kospi_info, kospi_all, on='Symbol')
kosdaq_all = pd.merge(kosdaq_info, kosdaq_all, on='Symbol')

In [15]:
# '상장연도' 칼럼의 데이터 타입을 정수형으로 변환
kospi_all['상장연도'] = kospi_all['상장연도'].astype(int)

# '회계년' 칼럼이 '상장연도' 칼럼보다 작은 행을 필터링하여 삭제
kospi_all = kospi_all[kospi_all['회계년'] >= kospi_all['상장연도']]

# 위 코드는 '회계년' 칼럼이 '상장연도' 칼럼보다 크거나 같은 행만 유지하고, 나머지 행은 모두 삭제합니다. 
# 이렇게 하면 '회계년'이 '상장연도'보다 작은 행은 필터링되어 제거됩니다.

In [16]:
# '상장연도' 칼럼의 데이터 타입을 정수형으로 변환
kosdaq_all['상장연도'] = kosdaq_all['상장연도'].astype(int)

# '회계년' 칼럼이 '상장연도' 칼럼보다 작은 행을 필터링하여 삭제
kosdaq_all = kosdaq_all[kosdaq_all['회계년'] >= kosdaq_all['상장연도']]

# 위 코드는 '회계년' 칼럼이 '상장연도' 칼럼보다 크거나 같은 행만 유지하고, 나머지 행은 모두 삭제합니다. 
# 이렇게 하면 '회계년'이 '상장연도'보다 작은 행은 필터링되어 제거됩니다.

In [17]:
print(kospi_all.shape, 
      kospi_all.drop_duplicates('Name_y').shape, 
      kosdaq_all.shape, 
      kosdaq_all.drop_duplicates('Name_y').shape)

# 코스피: 8461개 / 779개 기업
# 코스닥: 13268개 / 1538개 기업

(7833, 48) (779, 48) (12447, 48) (1534, 48)


In [18]:
kospi_all.drop(['Name_x'], axis=1, inplace=True)
kospi_all.rename(columns={'Name_y': 'Name'}, inplace=True)
kosdaq_all.drop(['Name_x'], axis=1, inplace=True)
kosdaq_all.rename(columns={'Name_y': 'Name'}, inplace=True)

---
### 금융업 제외시키기 (불필요한 데이터 제거)
- WICS 기준 금융업 제외
- 은행 / 증권 / 다각화된금융 / 보험 / 부동산
- 키워드 제거

In [19]:
# 은행
bank = ['KB금융', 
        '신한지주', 
        '하나금융지주', 
        '우리금융지주', 
        '기업은행', 
        '카카오뱅크', 
        'BNK금융지주', 
        'JB금융지주', 
        'DGB금융지주', 
        '상상인', 
        '제주은행']

# result = result[~result['Name'].isin(bank)]

# 증권
securities = ['메리츠금융지주', 
              '한국금융지주', 
              '미래에셋증권', 
              '삼성증권', 
              'NH투자증권', 
              '키움증권', 
              '다우기술', 
              '대신증권', 
              '한화투자증권', 
              '유진투자증권', 
              '신영증권', 
              '유안타증권', 
              'SK증권', 
              '다우데이타', 
              '다올투자증권', 
              '교보증권', 
              '현대차증권', 
              'DB금융투자', 
              '케이프', 
              '한양증권', 
              '부국증권', 
              '이베스트투자증권', 
              '유화증권', 
              '상상인증권', 
              '코리아에셋투자증권']

# result = result[~result['Name'].isin(securities)]

# 다각화된금융
diversified = ['삼성카드', 
               '우리기술투자', 
               '스틱인베스트먼트', 
               '글로벌텍스프리', 
               '미래에셋벤처투자', 
               '아주IB투자', 
               '에이티넘인베스트', 
               '리드코프', 
               'DSC인베스트먼트', 
               'SV인베스트먼트', 
               'SBI인베스트먼트', 
               '대성창투', 
               '한국캐피탈', 
               "메이슨캐피탈", 
               'HB인베스트먼트', 
               '캡스톤파트너스', 
               '스톤브릿지벤처스', 
               '컴퍼니케이', 
               '나우IB', 
               '큐캐피탈', 
               'TS인베스트먼트', 
               '푸른저축은행', 
               'LB인베스트먼트', 
               '리더스 기술투자', 
               '린드먼아시아']

# result = result[~result['Name'].isin(diversified)]

insurance = ['삼성화재', 
             '삼성생명', 
             'DB손해보험', 
             '현대해상', 
             '한화생명', 
             '코리안리', 
             '한화손해보험', 
             '미래에셋생명', 
             '롯데손해보험', 
             '동양생명', 
             '인카금융서비스', 
             '에이플러스에셋', 
             '흥국화재']

# result = result[~result['Name'].isin(insurance)]

kospi_all = kospi_all[~kospi_all['Name'].isin(set(bank)|set(securities)|set(diversified)|set(insurance))]
kosdaq_all = kosdaq_all[~kosdaq_all['Name'].isin(set(bank)|set(securities)|set(diversified)|set(insurance))]

In [20]:
kospi_all = kospi_all[~kospi_all['Name'].str.contains('은행|증권|투자|인베스트먼트|스팩|보험|화재|신탁|금융|카드|캐피탈|우리종금|대성창투')]
kosdaq_all = kosdaq_all[~kosdaq_all['Name'].str.contains('은행|증권|투자|인베스트먼트|스팩|보험|화재|신탁|금융|카드|캐피탈|우리종금|대성창투')]
# 생명의 경우 -> JW생명과학, 진원생명과학, LG생명과학도 필터링됨 -> 일단 보류

In [21]:
print(kospi_all.shape, 
      kospi_all.drop_duplicates('Name').shape, 
      kosdaq_all.shape, 
      kosdaq_all.drop_duplicates('Name').shape)

# 코스피: 7952개 / 733개 기업
# 코스닥: 12994개 / 1472개 기업

(7363, 47) (733, 47) (12188, 47) (1468, 47)


---
### 코로나 제외해 보자 (일단 패스)

In [ ]:
# kospi_excepted = kospi_all[kospi_all['회계년'].isin(list(range(2012, 2020)))]
# kosdaq_excepted = kosdaq_all[kosdaq_all['회계년'].isin(list(range(2012, 2020)))]

# print(kospi_excepted.shape, 
#       kospi_excepted.drop_duplicates('Name').shape, 
#       kosdaq_excepted.shape, 
#       kosdaq_excepted.drop_duplicates('Name').shape)

# # 코스피: 5188개 / 701개 기업
# # 코스닥: 8017개 / 1251개 기업

---
### 현금흐름/총차입부채(%) 칼럼 결측치 가지는 기업 명단

In [22]:
import numpy as np
kospi_all['현금흐름/총차입부채(%)'].replace('N/A(IFRS)', np.nan, inplace=True)
a = kospi_all.loc[kospi_all['현금흐름/총차입부채(%)'].isna()]
a['Name'].unique()

array(['BGF리테일', 'BGF', 'CS홀딩스', 'DL이앤씨', 'F&F', 'F&F홀딩스', 'HDC',
       'HDC현대산업개발', 'HL만도', 'HS애드', 'KISCO홀딩스', 'KPX홀딩스', 'LG', 'LS에코에너지',
       'NHN', 'SJM', 'SJM홀딩스', 'SK스퀘어', 'SNT다이내믹스', 'SNT모티브', 'SNT에너지',
       'SNT홀딩스', 'WISCOM', '강남제비스코', '강원랜드', '갤럭시아에스엠', '경동도시가스',
       '경동인베스트', '경인전자', '계양전기', '광주신세계', '남양유업', '넷마블', '대교', '대구백화점',
       '대덕', '대덕전자', '대웅', '대원제약', '대현', '덕양산업', '동방아그로', '동부건설', '동서',
       '동아에스티', '동아타이어', '동양', '동일고무벨트', '동일산업', '동화약품', '두산퓨얼셀',
       '롯데관광개발', '롯데웰푸드', '모토닉', '무학', '미래아이앤지', '미원에스씨', '미창석유', '부광약품',
       '부국철강', '비비안', '비상교육', '비케이탑스', '빙그레', '삼성E&A', '삼성공조', '삼성출판사',
       '삼영무역', '삼원강재', '샘표', '샘표식품', '서연', '서연이화', '성보화학', '성창기업지주',
       '세아제강', '세우글로벌', '솔루스첨단소재', '시디즈', '신도리코', '신세계 I&C', '신세계푸드',
       '신송홀딩스', '아모레G', '아모레퍼시픽', '아세아', '아세아시멘트', '아시아나IDT', '아이마켓코리아',
       '애경케미칼', '에스원', '에어부산', '에이블씨엔씨', 'HDC랩스', 'HD현대건설기계', 'HD현대',
       'HD현대일렉트릭', '엔씨소프트', '엘브이엠씨홀딩스', 'LX홀딩스', '영보화학', '영원무역홀딩스', '오리온',
       '

In [23]:
kosdaq_all['현금흐름/총차입부채(%)'].replace('N/A(IFRS)', np.nan, inplace=True)
b = kosdaq_all.loc[kosdaq_all['현금흐름/총차입부채(%)'].isna()]
b['Name'].unique()

array(['CMG제약', 'ES큐브', 'MDS테크', '강원에너지', '고려시멘트', '고려신용정보', '고영', '골프존',
       '국보디자인', '국순당', '국전약품', '글로벌에스엠', '글로본', '금강철강', '금화피에스시', '나무기술',
       '나스미디어', '나이스디앤비', '남화산업', '남화토건', '네오셈', '네오위즈', '네오위즈홀딩스', '네이블',
       '넥스턴바이오', '넥스트아이', '노바렉스', '노바텍', '농우바이오', '누보', '다보링크', '대동스틸',
       '대명소노시즌', '대산F&B', '대아티아이', '대양전기공업', '대원미디어', '대원산업',
       'THE MIDONG', '더블유에스아이', '덕산네오룩스', '덕우전자', '데브시스터즈', '덴티스', '동방선기',
       '동아엘텍', '동양파일', '동일금속', '동일기연', '듀오백', '드림시큐리티', '디모아', '디앤씨미디어',
       'DXVX', '디와이디', '디이엔티', '디지털대성', '디지틀조선', '딜리', '러셀', '로보로보',
       '로스웰', '로지시스', '룽투코리아', '리노공업', '매일유업', '매일홀딩스', '멀티캠퍼스', '메가스터디',
       '메가스터디교육', '메가엠디', '메디아나', '메디포스트', '메디프론', '메지온', '모두투어', '모베이스',
       '모코엠시스', '모헨즈', '미래생명자원', '바이오솔루션', '밸로프', '버킷스튜디오', '본느', '부스타',
       '부방', '뷰웍스', '브리지텍', '블레이드 Ent', '블루콤', '비아트론', 'BF랩스', '모아라이프플러스',
       '비즈니스온', '비투엔', '빅솔론', '빛샘전자', '사람인', '삼목에스폼', '삼아제약', '삼일기업공사',
       '삼진', '삼현철강', '삼화네트웍스', '서산', '서암기계공업', '서울리거', '서원

---
### 현금흐름/총차입부채(%) 칼럼 지우기

In [24]:
kospi_all.drop('현금흐름/총차입부채(%)', axis=1, inplace=True)
kosdaq_all.drop('현금흐름/총차입부채(%)', axis=1, inplace=True)

---
### 결측치 / 무한대 값 처리
- 결측치(N/A(IFRS) 포함) 제거
- 무한대 값 (inf/-inf) -> 결측치 -> 제거

In [25]:
kospi_all.replace('N/A(IFRS)', np.nan, inplace=True)

In [26]:
kospi_all.isna().sum()

Symbol                 0
상장연도                   0
Name                   0
회계년                    0
매출총이익률(%)             19
영업이익률(%)              19
당기순이익률(%)             19
자본금영업이익률(%)           57
영업수익/영업비용(%)          11
세전계속사업이익률(%)          19
자본금세전계속사업이익률(%)       57
자본금지배주주순이익률(%)        57
매출액증가율(전년동기)(%)       66
영업이익증가율(전년동기)(%)      57
당기순이익증가율(전년동기)(%)     57
총자본증가율(전년동기)(%)       57
유형자산증가율(전년동기)(%)      77
부채비율(%)               11
유동비율(%)               11
당좌비율(%)               11
현금비율(%)               11
유형자산구성비율(%)           11
무형자산구성비율(%)           11
판관비율(%)               11
타인자본비율(%)             11
총자산회전율(회)             57
매출채권회전율               57
재고자산회전율               57
총부채회전율(회)             57
총자본회전율(회)             57
자본금회전율(회)             57
고정자산회전율               46
현금흐름/총부채(%)           57
현금흐름/영업수익(%)          19
현금흐름/총자본(%)           57
영업/투자 현금흐름 비율         11
DSRI                 307
GMI                   70
AQI                   57
SGI                   58


In [27]:
kospi_all.isin([np.inf, -np.inf]).sum()

Symbol                 0
상장연도                   0
Name                   0
회계년                    0
매출총이익률(%)              0
영업이익률(%)               0
당기순이익률(%)              0
자본금영업이익률(%)            0
영업수익/영업비용(%)           0
세전계속사업이익률(%)           0
자본금세전계속사업이익률(%)        0
자본금지배주주순이익률(%)         0
매출액증가율(전년동기)(%)        0
영업이익증가율(전년동기)(%)       0
당기순이익증가율(전년동기)(%)      0
총자본증가율(전년동기)(%)        0
유형자산증가율(전년동기)(%)       5
부채비율(%)                0
유동비율(%)                0
당좌비율(%)                0
현금비율(%)                0
유형자산구성비율(%)            0
무형자산구성비율(%)            0
판관비율(%)                8
타인자본비율(%)              0
총자산회전율(회)              0
매출채권회전율              246
재고자산회전율              699
총부채회전율(회)              0
총자본회전율(회)              0
자본금회전율(회)              0
고정자산회전율                0
현금흐름/총부채(%)            0
현금흐름/영업수익(%)           0
현금흐름/총자본(%)            0
영업/투자 현금흐름 비율          0
DSRI                  40
GMI                    0
AQI                    0
SGI                    8


In [28]:
# 무한대 값을 결측치로 대체
kospi_all.replace([np.inf, -np.inf], 0, inplace=True)

# 결측치를 가지는 행 제거
kospi_all.dropna(axis=0, inplace=True)

In [29]:
kospi_all.isna().sum()

Symbol               0
상장연도                 0
Name                 0
회계년                  0
매출총이익률(%)            0
영업이익률(%)             0
당기순이익률(%)            0
자본금영업이익률(%)          0
영업수익/영업비용(%)         0
세전계속사업이익률(%)         0
자본금세전계속사업이익률(%)      0
자본금지배주주순이익률(%)       0
매출액증가율(전년동기)(%)      0
영업이익증가율(전년동기)(%)     0
당기순이익증가율(전년동기)(%)    0
총자본증가율(전년동기)(%)      0
유형자산증가율(전년동기)(%)     0
부채비율(%)              0
유동비율(%)              0
당좌비율(%)              0
현금비율(%)              0
유형자산구성비율(%)          0
무형자산구성비율(%)          0
판관비율(%)              0
타인자본비율(%)            0
총자산회전율(회)            0
매출채권회전율              0
재고자산회전율              0
총부채회전율(회)            0
총자본회전율(회)            0
자본금회전율(회)            0
고정자산회전율              0
현금흐름/총부채(%)          0
현금흐름/영업수익(%)         0
현금흐름/총자본(%)          0
영업/투자 현금흐름 비율        0
DSRI                 0
GMI                  0
AQI                  0
SGI                  0
DEPI                 0
SGAI                 0
LVGI                 0
TATA       

In [30]:
print(kospi_all.shape, 
      kospi_all.drop_duplicates('Name').shape)

# 코스피: 7045개 / 713개 기업

(7045, 46) (713, 46)


In [31]:
kosdaq_all.replace('N/A(IFRS)', np.nan, inplace=True)

In [32]:
kosdaq_all.isna().sum()

Symbol                 0
상장연도                   0
Name                   0
회계년                    0
매출총이익률(%)            108
영업이익률(%)             108
당기순이익률(%)            108
자본금영업이익률(%)          145
영업수익/영업비용(%)          79
세전계속사업이익률(%)         108
자본금세전계속사업이익률(%)      145
자본금지배주주순이익률(%)       145
매출액증가율(전년동기)(%)      143
영업이익증가율(전년동기)(%)     124
당기순이익증가율(전년동기)(%)    124
총자본증가율(전년동기)(%)      124
유형자산증가율(전년동기)(%)     130
부채비율(%)               80
유동비율(%)               89
당좌비율(%)               89
현금비율(%)               89
유형자산구성비율(%)           80
무형자산구성비율(%)           80
판관비율(%)               79
타인자본비율(%)             80
총자산회전율(회)            145
매출채권회전율              155
재고자산회전율              157
총부채회전율(회)            145
총자본회전율(회)            145
자본금회전율(회)            145
고정자산회전율               67
현금흐름/총부채(%)          145
현금흐름/영업수익(%)         108
현금흐름/총자본(%)          145
영업/투자 현금흐름 비율         79
DSRI                 220
GMI                  181
AQI                  135
SGI                  143


In [33]:
kosdaq_all.isin([np.inf, -np.inf]).sum()

Symbol                  0
상장연도                    0
Name                    0
회계년                     0
매출총이익률(%)               0
영업이익률(%)                0
당기순이익률(%)               0
자본금영업이익률(%)             0
영업수익/영업비용(%)            0
세전계속사업이익률(%)            0
자본금세전계속사업이익률(%)         0
자본금지배주주순이익률(%)          0
매출액증가율(전년동기)(%)        18
영업이익증가율(전년동기)(%)        3
당기순이익증가율(전년동기)(%)       2
총자본증가율(전년동기)(%)         0
유형자산증가율(전년동기)(%)        8
부채비율(%)                 0
유동비율(%)                 0
당좌비율(%)                 0
현금비율(%)                 0
유형자산구성비율(%)             0
무형자산구성비율(%)             0
판관비율(%)                27
타인자본비율(%)               0
총자산회전율(회)               0
매출채권회전율                59
재고자산회전율              1223
총부채회전율(회)               0
총자본회전율(회)               0
자본금회전율(회)               0
고정자산회전율                 0
현금흐름/총부채(%)             0
현금흐름/영업수익(%)            0
현금흐름/총자본(%)             0
영업/투자 현금흐름 비율           0
DSRI                   61
GMI                     0
AQI         

In [34]:
# 무한대 값을 결측치로 대체
kosdaq_all.replace([np.inf, -np.inf], 0, inplace=True)

# 결측치를 가지는 행 제거
kosdaq_all.dropna(axis=0, inplace=True)

In [35]:
kosdaq_all.isna().sum()

Symbol               0
상장연도                 0
Name                 0
회계년                  0
매출총이익률(%)            0
영업이익률(%)             0
당기순이익률(%)            0
자본금영업이익률(%)          0
영업수익/영업비용(%)         0
세전계속사업이익률(%)         0
자본금세전계속사업이익률(%)      0
자본금지배주주순이익률(%)       0
매출액증가율(전년동기)(%)      0
영업이익증가율(전년동기)(%)     0
당기순이익증가율(전년동기)(%)    0
총자본증가율(전년동기)(%)      0
유형자산증가율(전년동기)(%)     0
부채비율(%)              0
유동비율(%)              0
당좌비율(%)              0
현금비율(%)              0
유형자산구성비율(%)          0
무형자산구성비율(%)          0
판관비율(%)              0
타인자본비율(%)            0
총자산회전율(회)            0
매출채권회전율              0
재고자산회전율              0
총부채회전율(회)            0
총자본회전율(회)            0
자본금회전율(회)            0
고정자산회전율              0
현금흐름/총부채(%)          0
현금흐름/영업수익(%)         0
현금흐름/총자본(%)          0
영업/투자 현금흐름 비율        0
DSRI                 0
GMI                  0
AQI                  0
SGI                  0
DEPI                 0
SGAI                 0
LVGI                 0
TATA       

In [36]:
print(kosdaq_all.shape, 
      kosdaq_all.drop_duplicates('Name').shape)

# 코스닥: 11931개 / 1449개 기업

(11931, 46) (1449, 46)


In [37]:
# <<최종>>
# 코스피: 7045개 / 713개 기업
# 코스닥: 11931개 / 1449개 기업

---
### 다듬기

In [ ]:
kospi_all.columns

In [ ]:
kospi_all.rename(columns ={'매출채권회전율':'매출채권회전율(회)', 
                           '재고자산회전율':'재고자산회전율(회)', 
                           '고정자산회전율':'고정자산회전율(회)', 
                           '영업/투자 현금흐름 비율':'영업현금흐름/투자현금흐름(%)'}, inplace=True)

In [ ]:
kosdaq_all.rename(columns ={'매출채권회전율':'매출채권회전율(회)', 
                            '재고자산회전율':'재고자산회전율(회)', 
                            '고정자산회전율':'고정자산회전율(회)', 
                            '영업/투자 현금흐름 비율':'영업현금흐름/투자현금흐름(%)'}, inplace=True)

In [ ]:
kospi_all = kospi_all.round(2)
kosdaq_all = kosdaq_all.round(2)

In [ ]:
kospi_all.dtypes

In [ ]:
# 'Symbol', 'Name', '회계년' 칼럼을 제외하고 모든 칼럼 내 데이터를 숫자형으로 변경
for column in kospi_all.columns:
    if column not in ['Symbol', 'Name', '회계년']:
        if kospi_all[column].dtype == 'object':  # 문자열 열인 경우에만 처리
            kospi_all[column] = kospi_all[column].str.replace(',', '')
            kospi_all[column] = kospi_all[column].astype(float)
        
# # 'Symbol', 'Name', '회계년' 칼럼 내 데이터를 문자형으로 변경
# for column in ['Symbol', 'Name', '회계년']:
#     kospi_all[column] = kospi_all[column].astype('str')

In [ ]:
# 'Symbol', 'Name', '회계년' 칼럼을 제외하고 모든 칼럼 내 데이터를 숫자형으로 변경
for column in kosdaq_all.columns:
    if column not in ['Symbol', 'Name', '회계년']:
        if kosdaq_all[column].dtype == 'object':  # 문자열 열인 경우에만 처리
            kosdaq_all[column] = kosdaq_all[column].str.replace(',', '')
            kosdaq_all[column] = kosdaq_all[column].astype(float)
        
# # 'Symbol', 'Name', '회계년' 칼럼 내 데이터를 문자형으로 변경
# for column in ['Symbol', 'Name', '회계년']:
#     kosdaq_all[column] = kosdaq_all[column].astype('str')

---
### csv로 저장

In [ ]:
kospi_all.to_csv('./정제 데이터/코스피.csv', index=False)
kosdaq_all.to_csv('./정제 데이터/코스닥.csv', index=False)